# **(ADD HERE THE NOTEBOOK NAME)**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1


My plan for this notebook is to take notes that I can refer to about different steps, I guess I already did this for the data collection notebook in the playground a bit.

So I will take notes on the different steps and maybe copy some python commands but I won't execute anything.


Section 2 Data study

this is the data understanding step

They use ydata_profiling to make a ProfileReport
```
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()
```

They one hot encode the categorical variables then do a correlation study with both pearson and spearman. They pick the top 5 correlated variables. They sort by absolute value of the correlation.

Then they create a df that just uses these features and the target

```
df_eda = df.filter(vars_to_study + ['Churn'])
df_eda.head(3)
```

They write some code to loop through and plot different things depending on the type of data, categorical vs numerical. vars to study is the list of top 5 correlated features.



In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


def plot_categorical(df, col, target_var):

    plt.figure(figsize=(12, 5))
    sns.countplot(data=df, x=col, hue=target_var, order=df[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


def plot_numerical(df, col, target_var):
    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=col, hue=target_var, kde=True, element="step")
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


target_var = 'Churn'
for col in vars_to_study:
    if df_eda[col].dtype == 'object':
        plot_categorical(df_eda, col, target_var)
        print("\n\n")
    else:
        plot_numerical(df_eda, col, target_var)
        print("\n\n")


NameError: name 'vars_to_study' is not defined

They do some feature engineering. They put one of the statistics into bins. I am not sure if this is relevant for me. They did this because most of the features they were looking at were categorical and not numerical. Since most of mine, all?, are numerical I don't think this will be relevant at all.
They then did a parallel plot, I forget what the analogue of this is for numerical data, maybe a pair plot?

This then seems to be sufficient analysis for answering the first business concern: which statistics are predictors of things. I think I was going to use predictive power and correlation. So that would be done first to isolate the most predictive features and then you plot the most predictive features against the target to see what the profile is of the target in terms of these features.

---

# Section 3

Now they are doing cleaning and imputation.

They use the following link https://docs.google.com/spreadsheets/d/1Pa04ng4d8tjBizKsUCUiUTH-eUNQunSN/edit?gid=1633765441#gid=1633765441

So I guess I should make a spreadsheet with rows for the features and columns like missing data, data type, percentage of data (not sure what this one means), and then description of how I am going to clean wrpt that feature.

I guess it woouldn't need to be a google spreadsheet, it could be something I include in the readme, or in a notebook.

So they plot the distribution of the features with missing data. I think I will probably drop the missing data, since I think it is from older games and can't really be properly imputed since levels of various statistics changed over the years (like )
```
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data
```
They then do a report on all the vars with missing data using the ydata thing.

Then they have this custom code for displaying correlation and pps
```
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)
```

There is very little cleaning. I don't think I will have much cleaning either.

maybe there will be certain features that I drop because they are the target.

They do the train test split before they clean the data.

since they remove cols they don't reassess the distribution, I will be removing rows, so I should reassess the distributions.

double check to see if there is more missing data



# Section 4?

feature engineering notebook

they also have a spreadsheet for different potential transformations regarding each feature that is still there. They encode the categorical data, I guess there is some of that for me like the team

there are some custom functions, I pasted them into utils

smart correlation (feature selection thing) should always be last.

we want to try and transform the numerical data so that it looks normally distributed.

After you do the transformations you need to evaluate how they did. For categorical, this is sort of pointless, since it just works. For the transformations like rescaling with log etc, this is relevant, you want to pick the one that is closest to a normal distribution shape. So I guess that is why we have that step.

Then we create the relevant encoders and apply them to the training set and the test set.

In the walkthrough, they didn't apply any numerical transformations.

then comes smart correlation. we don't need to list which variables we will be dealing with here.

This then gives us the steps to add to the pipeline., or the first half of them I guess.



# Section 5
modelling and pipeline etc.

they load the data and drop the some columns. These are columns that are the targets of the different algorithms, as well as the features that were deemed not suitable for other reasons (this is where I would remove things like name or whatever.)

Then they build the pipeline. first encoding categorical data. They would transform the numerical data to be more of a normal distribution if they had found a transformation that did this. Similar to outliers. Then doing the correlation sampler thing where redundant features are removed.

They then get to other parts of the pipeline. They didn't just do the smart correlation selection, they are also going to do the feature selection from scikit learn.

They then did scaling and select from model.
They loaded a bunch of different algorithms from sklearn and are going to test each of them.

Remember that if I use a neural network I won't need to remove correlated things etc.



In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")

            model = PipelineClf(self.models[key])
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches


The above is stuff for doing hyper parameter selection. I guess I can use it. I can also tinker with it.

Then you do the test train split.

I guess there are sort of two pipelines, a cleaning and data engineering one, and we fit that pipeline to X_train, then transfrom both X_train and X_test (for X_train, we pass it to the fit_transform method of the pipeline.)

There is also the classification pipeline, which has the scalar and feature selection in it. This is weird, maybe ask mo about it?

I won't be doing any stuff with imbalanced datasets I don't think.

Then the gridsearch cross validation begins. So you have to make a bunch of pipelines by passing models to them and then also making dictionaries for the parameters, these were all empty...? I think this is because we are doing the hyper parameters later.

yeah, they do the hyper parameters after they have picked which model they are going to do, I don't know if I will do it that way.

There is a thing that happens where several choices of hyper parameters seem to perform the same, what are we supposed to do then?

Remember to check the performance against the metrics set out at the beginning. We also want the performance on the training and test set to be similar. if it is much better on the training set then that is an overfit.

They then go back and rebuild the pipeline? They do this so that in production the pipeline only has the most important features. I am not sure how I will address this.

then retrain the model on the best features with the desired parameters. Since we are only using the best features they drop the feature sleection steps of the pipeline.

Then they save the data appropriately.

# Section I forget which number 7?

Now they move on to predicting tenure. So this is a regression problem.

Again, they use ordinal encoder, smart correlation selection, standard scaler, select from model, used some tree based regression models, and a linear regression.

they only care about predicting tenure for users that will churn, what is the analogue of this with my problem?

here they only have one pipeline, why? they say it is because they have to address target imbalance.

Then they get the best features with:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# after data cleaning and feature engineering, the features may have changes
# how many data cleaning and feature engineering steps does your pipeline have?
data_cleaning_feat_eng_steps = 2
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()


Since the performance was so poor, they say we should try doing PCA instead of feature selection.

They transform the data (scale it etc) and then pass it to some pca thing.

They get at least 80% of the variance with 7 components, and then they rebuild the pipeline using 7 components for pca (this explains 72% of the variance, and is just an initial trial, I guess), and they put that in place of the select from model feature selection step. they still use the smart selection correlation thing. Then they redo the grid search to determine the best model and then best hyper parameters.

We still are not meeting the requirements from the business case step. So ideally, we would spend more time tuning the hyperparameters. When this still doesn't work, then we can convert the ml task from regression to classification by using bins.

since we change from regression to classification, we have to talk to stakeholders about new thresholds for success. because of the business case, they wanted to know users that will churn in the short term so we care about recall on the 4month or less class. The model did meet our requirements, but shit the bed on others.





# Section 8 Cluster Analysis
They make a pipeline for doing cluster analysis.

In their pipeline, they have the ordinal encoder, smart correlation selection, scaler, PCA, and then a Kmeans unsupervised model. They truncate the pipeline up to PCA (and including) to figure out how many components they should take. You always start with the same number of components as there are features, at least. Then they update the hyperparameter in the PCA step.

Also, they start with the n_clusters=50 as the hyperparam for kmeans model. Then Look at the elbow and do silhouette scores. Pick the number of clusters with a good (highest avg maybe?) silhouette score. The we fit the pipeline and add a col to the df for cluster. We then decide to try and fit a classifier to predict the cluster. We are reverse engineering what the profiles of the clusters are by doing the classification, and then looking at the important features for that classification pipeline.

lots of custom functions here for displaying stuff. I guess I will just go back to the notebook to get the code if I need/want it.

once you have the important features for classifying the clusters, then you redo the clustering model where you only have those features. You need redo all of the analysis and see if it is about the same as the previous clustering model with all of the features. We want to reduce the number of features for performance (like computationally?) reasons, but we don't want to lose quality. Then we have to do the classification model again to see if the clusters we get form the new algorithm have the same profile (not identical, but comparable) and the same important features.

In the example, the clustering with fewer features was improved in fact.

Then save the figures and the models etc.



# Section 9?
Developing the dashboard.

Come up with a plan for the dashboard, like how many pages and what goes on each page.

- Intro page
    - Should explain project goals like hypothesis and business case
    - explain special terms for data
    - give opportunity to look at data
    - give reference to data
    - summary from a notebook showing what we found in the correlation study, I guess?
    - then there should be some graphs, not sure what kind to have. they had a parallel plot and churn distributed with respect to the different correlated variables.
- page to make predictions
    - not sure how I should do this. I could just have them select games from the most recent season which I do not train my data on.
- project hypothesis and validation page, just text explaining things
- pages with reports
    - one for each pipeline
    - blurb explaining the conclusions related to that pipeline
    - report with metrics related to that pipeline, like classification report stuff and maybe some graphs
- pages have a similar structure, code wise
    - load data from where it was saved, like df, model, and figures
    - write some general conclusioins we learned form the model
    - present some graphics and or data that support this
    - also explain what the pipeline is in each case
    

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
